In [ ]:
!pip install tensorflow openvino-dev

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import tensorflow as tf
from tensorflow import keras
import joblib

In [2]:
# Load CSV file (exported from InfluxDB)
df = pd.read_csv("./data/battery_data.csv")

# Convert timestamp to datetime for time-series processing
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Simulate Time-to-Failure (assuming failure happens at the last recorded timestamp)
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600  # Convert to hours

# Display processed data
print(df.head())

                         timestamp  batteryId  ambientTemp  batteryCurrent  \
0 2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1 2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2 2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3 2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4 2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   

   batteryTemp  batteryVoltage  distance     kmh  stateOfCharge  \
0        25.22          396.39      0.20  127.75         0.9991   
1        25.33          396.23      0.31  130.46         0.9986   
2        25.39          396.49      0.42  125.59         0.9981   
3        25.44          396.66      0.52  122.42         0.9977   
4        25.47          396.58      0.62  123.87         0.9972   

   stateOfHealth  timeBeforeFailure  
0        99.9998           0.425357  
1        99.9997           0.424569  
2        99.99

In [3]:
# Select Features & Target Variable
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define neural network
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1)  # Regresión -> una sola salida
])

# Compile model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [ ]:
# Train model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

In [5]:
# Make Predictions
y_pred = model.predict(X_test_scaled).flatten()

In [6]:
# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.2f} hours before failure")

Model MAE: 0.02 hours before failure


In [ ]:
# Save model as keras and scaler for inference
model.save("models/battery_ttf_model.keras")
joblib.dump(scaler, "models/scaler.pkl")
print("Model saved as TensorFlow format in 'models/battery_ttf_model_tf.keras'")

In [ ]:
# Load model
model = tf.keras.models.load_model("models/battery_ttf_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_ttf_model")

print("Model converted to SavedModel format")

In [ ]:
!mo --saved_model_dir models/battery_ttf_model --output_dir openvino_model